# Hub Time

Just run the below cell

In [ ]:
%gui asyncio
import ipywidgets as widgets
from datetime import datetime, timedelta
import asyncio
import os

slider = widgets.SelectionSlider(
    options=[f"{h}h" for h in range(1, 25)] + [f"{d}d" for d in range(1, 8)],
    value="6h",
    description="Time",
    disabled=False,
)

status_text = widgets.Text(value="", disabled=False)

def set_button(btn):
    if btn.description == "Start":
        btn.description = "Stop"
        btn.style.button_color = "lightcoral"
    else:
        btn.description = "Start"
        btn.style.button_color = "lightgreen"

def get_msg(diff: timedelta):

    minutes, sec = divmod(diff.seconds, 60)
    hours, minutes = divmod(minutes, 60)
    msg = f"Time left: "
    if diff.days:
        msg += f"{diff.days} days, "
    if hours:
        msg += f"{hours} hours, "

    msg += f"{minutes} minutes"
    return msg

async def wall_time():

    run_time = slider.value

    if run_time[-1] == "h":
        hours = int(run_time[:-1])
    elif run_time[-1] == "d":
        hours = int(run_time[:-1]) * 24
    else:
        raise ValueError("Invalid time format")

    start = datetime.now()
    end = start + timedelta(hours=hours)
    
    while datetime.now() < end:
        diff = end - datetime.now()
        print("Time left: ", diff)
        status_text.value = get_msg(diff)
        await asyncio.sleep(300)
    set_button(btn)
    status_text.value = "Time is up!"
    

btn = widgets.Button(description="Start")
btn.style.button_color = "lightgreen"

def wait_for_click(btn):
    
    if btn.description == "Start":
        task = asyncio.get_event_loop().create_task(name="wall_time", coro=wall_time())
    else:
        task, = [task for task in asyncio.all_tasks() if task.get_name() == "wall_time"]
        task.cancel()
        status_text.value = "Stopped"

    set_button(btn)


markdown_widget = widgets.HTML(
    value="""<h2 style='text-align: center;'>⏰ Hub time ⏰</h2>
    <p>This notebook is used to schedule your jupyter hub to run for a given interval.</p>
    <p>Simply select the amount of time you want the hub to run and click the start button.</p>
    <p>To start a long running notebook in the background use:</p>
    <code style='background-color: black; color: white;'>nohup jupyter nbconvert --execute  --allow-errors --to notebook --inplace [NOTEBOOK_NAME].ipynb &</code>
    </p>
    <p>Then you can close the browser and the notebook will continue running and storing the output</p>
    <p>You can of course use `nohup` to run other commands also</p>
    """,
)
stop_widget = widgets.HTML(f"""
<p style='text-align: center;font-style: italic;'>time updated every 5min</p>
<p>When you want to stop the notebook you can stop the kernel or press the <b>stop</b> button</p>
<p>Alternatively you can run the following command in a terminal:</p>
<code style='background-color: black; color: white;'>kill {os.getpid()}</code>
<h3 style='text-align: center;'>If the computation finish early <b>please</b> stop the server 🥺 </h3>
<p style='text-align: center;'>Thank you! 🙏</p>
<p></p>
<p><b>Keep this notebook open on the jupyterhub tab menu to view the counter from different browser sessions</b></p>
""")
btn.on_click(wait_for_click)
label = widgets.Label(value="Time left: ")
widgets.VBox(
    [markdown_widget, slider, btn, status_text, stop_widget],
    layout=widgets.Layout(align_items="center"),
)